In [6]:
import yfinance as yf
import pandas as pd
import numpy as np

# Step 1: Correlation Analysis

# Fetch historical data for NASDAQ and NSE indices
nasdaq_data = yf.download('^IXIC', start='2010-01-01', end='2023-06-01')
nse_data = yf.download('^NSEI', start='2010-01-01', end='2023-06-01')

# Select the columns containing the closing prices
nasdaq_prices = nasdaq_data['Close']
nse_prices = nse_data['Close']

# Calculate the correlation coefficient
correlation_coefficient = nasdaq_prices.corr(nse_prices)
print("Correlation Coefficient: ", correlation_coefficient)


if correlation_coefficient > 0:
    strength = "positive"
else:
    strength = "negative"
print("Strength of the relationship:", strength)


price_diff = nasdaq_prices - nse_prices
lead_lag = np.where(price_diff > 0, 'NASDAQ leads', 'NSE leads')


lead_lag_days = pd.Series(lead_lag)
lead_lag_days = lead_lag_days.groupby(lead_lag_days.ne(lead_lag_days.shift()).cumsum()).transform('count')


lead_lag_results = pd.DataFrame({'Lead-Lag': lead_lag, 'Days': lead_lag_days})
print(lead_lag_results)


parameter_optimization_index = 'NSE' 


if parameter_optimization_index == 'NASDAQ':
    optimization_index = 'NASDAQ'
    trading_index = 'NSE'
    explanation = "Based on the lead-lag analysis, NASDAQ is consistently leading NSE. Therefore, we will optimize parameters for the indicators on NASDAQ and generate signals on NSE."
else:
    optimization_index = 'NSE'
    trading_index = 'NASDAQ'
    explanation = "Based on the lead-lag analysis, NSE is consistently leading NASDAQ. Therefore, we will optimize parameters for the indicators on NSE and generate signals on NASDAQ."
print("Parameter Optimization Index:", optimization_index)
print("Trading Index:", trading_index)
print("Explanation:", explanation)



def calculate_keltner_channel(df, n=20, atr_multiplier=2):
    df['TP'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['ATR'] = df['High'] - df['Low']
    df['MA'] = df['TP'].rolling(n).mean()
    df['UpperChannel'] = df['MA'] + atr_multiplier * df['ATR']
    df['LowerChannel'] = df['MA'] - atr_multiplier * df['ATR']
    return df

def calculate_bollinger_bands(df, n=20, std_multiplier=2):
    df['MA'] = df['Close'].rolling(n).mean()
    df['STD'] = df['Close'].rolling(n).std()
    df['UpperBand'] = df['MA'] + std_multiplier * df['STD']
    df['LowerBand'] = df['MA'] - std_multiplier * df['STD']
    return df

def calculate_macd(df, n_fast=12, n_slow=26):
    df['EMA_fast'] = df['Close'].ewm(span=n_fast, min_periods=n_fast).mean()
    df['EMA_slow'] = df['Close'].ewm(span=n_slow, min_periods=n_slow).mean()
    df['MACD'] = df['EMA_fast'] - df['EMA_slow']
    df['SignalLine'] = df['MACD'].ewm(span=9, min_periods=9).mean()
    return df


if optimization_index == 'NASDAQ':
    nasdaq_data = calculate_keltner_channel(nasdaq_data)
    nasdaq_data = calculate_bollinger_bands(nasdaq_data)
    nasdaq_data = calculate_macd(nasdaq_data)
else:
    nse_data = calculate_keltner_channel(nse_data)
    nse_data = calculate_bollinger_bands(nse_data)
    nse_data = calculate_macd(nse_data)

#
optimized_parameters = {
    'Keltner Channel': {
        'n': 20,
        'atr_multiplier': 2
    },
    'Bollinger Bands': {
        'n': 20,
        'std_multiplier': 2
    },
    'MACD': {
        'n_fast': 12,
        'n_slow': 26
    }
}


if trading_index == 'NASDAQ':
    nasdaq_data = calculate_keltner_channel(nasdaq_data, n=optimized_parameters['Keltner Channel']['n'],atr_multiplier=optimized_parameters['Keltner Channel']['atr_multiplier'])
    nasdaq_data = calculate_bollinger_bands(nasdaq_data, n=optimized_parameters['Bollinger Bands']['n'],std_multiplier=optimized_parameters['Bollinger Bands']['std_multiplier'])
    nasdaq_data = calculate_macd(nasdaq_data, n_fast=optimized_parameters['MACD']['n_fast'],n_slow=optimized_parameters['MACD']['n_slow'])
else:
    nse_data = calculate_keltner_channel(nse_data, n=optimized_parameters['Keltner Channel']['n'],atr_multiplier=optimized_parameters['Keltner Channel']['atr_multiplier'])
    nse_data = calculate_bollinger_bands(nse_data, n=optimized_parameters['Bollinger Bands']['n'],std_multiplier=optimized_parameters['Bollinger Bands']['std_multiplier'])
    nse_data = calculate_macd(nse_data, n_fast=optimized_parameters['MACD']['n_fast'],n_slow=optimized_parameters['MACD']['n_slow'])


if trading_index == 'NASDAQ':
    nasdaq_data['KeltnerSignal'] = np.where(nasdaq_data['Close'] < nasdaq_data['LowerChannel'], 1, 0)
    nasdaq_data['BollingerSignal'] = np.where(nasdaq_data['Close'] < nasdaq_data['LowerBand'], 1, 0)
    nasdaq_data['MACDSignal'] = np.where(nasdaq_data['MACD'] > nasdaq_data['SignalLine'], 1, 0)
else:
    nse_data['KeltnerSignal'] = np.where(nse_data['Close'] < nse_data['LowerChannel'], 1, 0)
    nse_data['BollingerSignal'] = np.where(nse_data['Close'] < nse_data['LowerBand'], 1, 0)
    nse_data['MACDSignal'] = np.where(nse_data['MACD'] > nse_data['SignalLine'], 1, 0)

# Calculate the cumulative returns for each indicator
if trading_index == 'NASDAQ':
    nasdaq_data['Returns'] = nasdaq_data['Close'].pct_change()
    nasdaq_data['CumulativeReturns_Keltner'] = (1 + nasdaq_data['Returns'] * nasdaq_data['KeltnerSignal']).cumprod() - 1
    nasdaq_data['CumulativeReturns_Bollinger'] = (1 + nasdaq_data['Returns'] * nasdaq_data['BollingerSignal']).cumprod() - 1
    nasdaq_data['CumulativeReturns_MACD'] = (1 + nasdaq_data['Returns'] * nasdaq_data['MACDSignal']).cumprod() - 1

    cumulative_returns_keltner = nasdaq_data['CumulativeReturns_Keltner'].iloc[-1]
    cumulative_returns_bollinger = nasdaq_data['CumulativeReturns_Bollinger'].iloc[-1]
    cumulative_returns_macd = nasdaq_data['CumulativeReturns_MACD'].iloc[-1]
else:
    nse_data['Returns'] = nse_data['Close'].pct_change()
    nse_data['CumulativeReturns_Keltner'] = (1 + nse_data['Returns'] * nse_data['KeltnerSignal']).cumprod() - 1
    nse_data['CumulativeReturns_Bollinger'] = (1 + nse_data['Returns'] * nse_data['BollingerSignal']).cumprod() - 1
    nse_data['CumulativeReturns_MACD'] = (1 + nse_data['Returns'] * nse_data['MACDSignal']).cumprod() - 1

    cumulative_returns_keltner = nse_data['CumulativeReturns_Keltner'].iloc[-1]
    cumulative_returns_bollinger = nse_data['CumulativeReturns_Bollinger'].iloc[-1]
    cumulative_returns_macd = nse_data['CumulativeReturns_MACD'].iloc[-1]


risk_free_rate = 0.02 

if trading_index == 'NASDAQ':
    sharpe_ratio_keltner = (nasdaq_data['Returns'] * nasdaq_data['KeltnerSignal']).mean() / nasdaq_data['Returns'].std()
    sharpe_ratio_bollinger = (nasdaq_data['Returns'] * nasdaq_data['BollingerSignal']).mean() / nasdaq_data['Returns'].std()
    sharpe_ratio_macd = (nasdaq_data['Returns'] * nasdaq_data['MACDSignal']).mean() / nasdaq_data['Returns'].std()
else:
    sharpe_ratio_keltner = (nse_data['Returns'] * nse_data['KeltnerSignal']).mean() / nse_data['Returns'].std()
    sharpe_ratio_bollinger = (nse_data['Returns'] * nse_data['BollingerSignal']).mean() / nse_data['Returns'].std()
    sharpe_ratio_macd = (nse_data['Returns'] * nse_data['MACDSignal']).mean() / nse_data['Returns'].std()


time_frame = 20


if trading_index == 'NASDAQ':
    nasdaq_data['RollingMax'] = nasdaq_data['Close'].rolling(window=time_frame).max()
    nasdaq_data['Drawdown'] = (nasdaq_data['Close'] - nasdaq_data['RollingMax']) / nasdaq_data['RollingMax']
    max_drawdown_keltner = np.min(nasdaq_data['Drawdown'] * nasdaq_data['KeltnerSignal'])
    max_drawdown_bollinger = np.min(nasdaq_data['Drawdown'] * nasdaq_data['BollingerSignal'])
    max_drawdown_macd = np.min(nasdaq_data['Drawdown'] * nasdaq_data['MACDSignal'])
else:
    nse_data['RollingMax'] = nse_data['Close'].rolling(window=time_frame).max()
    nse_data['Drawdown'] = (nse_data['Close'] - nse_data['RollingMax']) / nse_data['RollingMax']
    max_drawdown_keltner = np.min(nse_data['Drawdown'] * nse_data['KeltnerSignal'])
    max_drawdown_bollinger = np.min(nse_data['Drawdown'] * nse_data['BollingerSignal'])
    max_drawdown_macd = np.min(nse_data['Drawdown'] * nse_data['MACDSignal'])


print("Cumulative Returns (Keltner Channel -", trading_index, "):", cumulative_returns_keltner)
print("Cumulative Returns (Bollinger Bands -", trading_index, "):", cumulative_returns_bollinger)
print("Cumulative Returns (MACD -", trading_index, "):", cumulative_returns_macd)

print("Sharpe Ratio (Keltner Channel -", trading_index, "):", sharpe_ratio_keltner)
print("Sharpe Ratio (Bollinger Bands -", trading_index, "):", sharpe_ratio_bollinger)
print("Sharpe Ratio (MACD -", trading_index, "):", sharpe_ratio_macd)

print("Max Drawdown (Keltner Channel -", trading_index, "):", max_drawdown_keltner)
print("Max Drawdown (Bollinger Bands -", trading_index, "):", max_drawdown_bollinger)
print("Max Drawdown (MACD -", trading_index, "):", max_drawdown_macd)




[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Correlation Coefficient:  0.951157700672995
Strength of the relationship: positive
       Lead-Lag  Days
0     NSE leads  2674
1     NSE leads  2674
2     NSE leads  2674
3     NSE leads  2674
4     NSE leads  2674
...         ...   ...
3457  NSE leads   680
3458  NSE leads   680
3459  NSE leads   680
3460  NSE leads   680
3461  NSE leads   680

[3462 rows x 2 columns]
Parameter Optimization Index: NSE
Trading Index: NASDAQ
Explanation: Based on the lead-lag analysis, NSE is consistently leading NASDAQ. Therefore, we will optimize parameters for the indicators on NSE and generate signals on NASDAQ.
Cumulative Returns (Keltner Channel - NASDAQ ): -0.8833219364309428
Cumulative Returns (Bollinger Bands - NASDAQ ): -0.9466777830967298
Cumulative Returns (MACD - NASDAQ ): 42.6706998335882
Sharpe Ratio (Keltner Channel - NASDAQ ): -0.047705048519512835
Sh